This file provides a few basic examples of Neet usage.

First let's load an example network.  There are a number of example networks in `neet.boolean.examples`.  The `s_pombe` network represents the interactions among 9 components that define the cell cycle of *S. pombe* (fission yeast).

In [1]:
import neet

In [2]:
from neet.boolean.examples import s_pombe

In [3]:
s_pombe.size

9

The most fundamental aspect of a Neet network is the `update` function, which defines the network's dynamics.  The `update` function takes a network state at timestep `t` and outputs the resulting state at `t+1`:

In [4]:
s_pombe.update([0,0,0,0,0,0,0,0,0])

[0, 1, 0, 0, 0, 0, 0, 0, 0]

The "informational architecture" of a Neet network encapsulates computation of quantities such as active information, entropy rate, transfer entropy, and mutual information.

In [5]:
from neet.information import Architecture

In [6]:
k = 2 # the history length, the size of each sample in information calculations
timesteps = 5 # the number of timesteps taken from all possible starting points to create samples
arch = Architecture(s_pombe, k, timesteps)

In [7]:
# the active information for each node
arch.active_information()

array([0.        , 0.06634219, 0.32303261, 0.32303261, 0.07918928,
       0.11236749, 0.24586519, 0.24586519, 0.02025117])

In [8]:
# the mutual information between a pair of nodes
node1index,node2index = 3,5
arch.mutual_information()[node1index,node2index]

0.009288397587760126

The average sensitivity measures the degree to which single bit flips lead to changes that spread through the system.

In [9]:
import neet.sensitivity as ns

In [10]:
ns.average_sensitivity(s_pombe)

0.9513888888888888

Neet can tell you which edges in the network are canalizing.

In [11]:
ns.canalizing_edges(s_pombe)

{(0, 0),
 (1, 2),
 (1, 3),
 (1, 4),
 (4, 5),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 6),
 (5, 7),
 (8, 4)}

If `networkx` and `pygraphviz` are installed, the `draw` function outputs an image representing the causal structure of the network.

In [12]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
plotly.offline.init_notebook_mode(connected=True)

import networkx as nx
%matplotlib inline


In [13]:
G = s_pombe.to_networkx_graph()

In [14]:
node_positions = nx.spring_layout(G)
node_positions = nx.kamada_kawai_layout(G)

In [15]:
nx.set_node_attributes(G, node_positions, 'pos')

In [16]:
pos=nx.get_node_attributes(G,'pos')
# pos = node_positions

dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

p=nx.single_source_shortest_path_length(G,ncenter)

In [17]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

In [18]:
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_info = '# of connections: '+str(len(adjacencies[1]))
    node_trace['text']+=tuple([node_info])

In [19]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)))

In [20]:
plotly.offline.plot(fig,filename="networkx")

/Users/Harrison/Library/Python/2.7/lib/python/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `networkx` didn't end with .html. Adding .html to the end of your file.



'networkx.html'

In [28]:
s_pombe.names

['SK',
 'Cdc2_Cdc13',
 'Ste9',
 'Rum1',
 'Slp1',
 'Cdc2_Cdc13_active',
 'Wee1_Mik1',
 'Cdc25',
 'PP']

In [56]:
s_pombe.draw(labels='names',prog='dot')

TypeError: draw() got an unexpected keyword argument 'prog'

In [22]:
landscape = neet.synchronous.Landscape(s_pombe)

In [23]:
landscape

In [24]:
L = landscape.graph

In [36]:
s_pombe.draw(labels='names')

In [31]:
nx.get_node_attributes(L,)

TypeError: get_node_attributes() takes exactly 2 arguments (1 given)

In [27]:
nx.nx_agraph.view_pygraphviz(L, prog='dot')#, path=filename)

('/var/folders/83/9fl54qvd0x31mgqsjq221w6m0000gn/T/tmpqaO1Ur.png',
 <AGraph <Swig Object of type 'Agraph_t *' at 0x10e7b2e70>>)

In [54]:
import sys
sys.path.insert(0,'../')#'~/Users/Harrison/Desktop/ASU/Neet/')
import neet
from neet.boolean.examples import s_pombe

In [55]:
s_pombe.draw()